In [5]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from collections import Counter
import random
from pprint import pprint
import pandas as pd
from dateutil import parser
import os
import json

In [6]:
def parseJournal(path):
    textLines = []
    with open(path,"r",encoding="UTF-8",errors="ignore") as txtFile:
        for line in txtFile:
            line= line.strip()
            textLines.append(line)
    commands = []
    fileName = "none"
    username = "none"
    for i, line in enumerate(textLines):
    #     matches = re.findall("(?<=Jrn.)(.[a-zA-Z]+)",line)
        line = re.sub("\s+"," ",line)
        matches = re.search("^Jrn",line)
    #     if len(matches) > 0:
        if matches:
            jrnName = re.findall("(?<=Jrn.)(.[a-zA-Z]+)",line)[0]
            commIndex = 0
            time = textLines[i-1]
            time = time.replace("'H ","")
            time = time.replace("'E ","")
            time = re.sub(";\s+\d+:<","",time)
            #GETTING COMMANDS
            
            command = line
            while re.search("_$",textLines[i+commIndex]):
                command += textLines[i+commIndex+1].replace("_","")
                commIndex +=1
            if jrnName == "Command":
                isCommand = True
                commandDetails = re.findall("[a-zA-Z\s\(\)\-\.]+\s,\s[a-zA-Z\s_]+|\s,\s[a-zA-Z\s_]+",command)
                commandDetails = commandDetails[0].split(" , ")

                if len(commandDetails) == 2:
                    commandDetails = {
                        "id":commandDetails[1],
                        "desc":commandDetails[0]
                    }
                else:
                    commandDetails = {
                        "id":commandDetails[0],
                        "desc":"none"
                    }
                if len(commandDetails) ==0:
                    print(command)
            else:
                isCommand = False
                commandDetails = {
                        "id":"none",
                        "desc":"none"
                    }
            details = re.findall("(?<=\")([a-zA-Z\s]+)(?=\")",line)
            if len(details)==0:
                details = ["none"]
            isWarning = False
            if jrnName == "Data" and "Mini warning dialog" in details:
                isWarning = True
                warrIndex = -2
                while re.search("^'\s1:<",textLines[i+warrIndex]):
                    warrIndex -= 1
                warningMessages = [textLines[i+warrIndex].replace("'","").strip()]
            elif jrnName == "Data" and "Error dialog" in details:
                isWarning = True
                warrIndex = -2
                warningMessages = []
                while re.search("^'\s1:<",textLines[i+warrIndex]):
                    if "summary" in textLines[i+warrIndex]:
                        break
                    if "Error" in textLines[i+warrIndex]:
                        warningMessage = textLines[i+warrIndex].replace("' 1:< Error: ","").replace("' 1:< Error","").strip()
                        warningMessage = re.sub("\s-\s\d\stimes","",warningMessage)
                        warningMessages.append(warningMessage)
                    warrIndex -= 1
            else:
                 warningMessages = ["none"]
            if jrnName == "Directive" and details[0] =="Username":
                username = re.findall('(?<=")[a-zA-Z0-9]+',command)[1]
            if jrnName == "Data" and "File Name" in details:
                dets = re.findall('[a-zA-Z\s\:\\0-9-\#\&\,\.]+\.rvt',command)
                if len(dets)>0:
                    fileName = dets[0]
            commands.append({"time":time,"entry":command,"jrnName":jrnName,"details":details[0],"commandID":commandDetails["id"],"commandDesc":commandDetails["desc"],"warningMessage":tuple(warningMessages),"isWarning":isWarning,"isCommand":isCommand,"fileName":fileName,"journalPath":path})
    df = pd.DataFrame.from_records(commands)
    df["username"] = username
    for command in commands:
        command["username"]= username
    return (df,commands)

In [7]:
#to get a journal file dataframe
# df = parseJournal("journal.0029.txt"[0]

In [8]:
#to get a journal file list of dicts
# df = parseJournal("journal.0029.txt"[1]

In [10]:
#put journal files in this folder
root_path = "./journalFiles/"
dfs = []
allCommands = []
for file in os.listdir(root_path):
    journalData = parseJournal(root_path+file)
    dfs.append(journalData[0])
    allCommands += journalData[1]
allDf = pd.concat(dfs)

In [ ]:
commands = df[df['isCommand']]
timeDiffs = []
for jp in commands["journalPath"].unique():
    jpSelect = commands[commands["journalPath"]==jp]
    for i,row in enumerate(jpSelect.iterrows()):
        if i == 0:
            timeDiffs.append(0)
        else:
            timeNow = row[1]["datetime"]
            timePrev = jpSelect.iloc[i-1]["datetime"]
            timeDif = timeNow - timePrev
            timeDiffs.append(timeDif.seconds)